In [1]:
# Import block

import alpha_shape_Mrestani as ash  # From Mrestani repo
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# Utility functions

def um_to_px(coords, pxum=6.25):
    # pxum is pixels per micron
    return coords*pxum - 0.5

In [3]:
# Load Nikon data file with points

dbscan_file = '20220526_L1_NMJ1_A2R67_DBSCAN_v2.xlsx'

#Read Excel file, extract columns of interest (**change out sheet name (B1_Ib_DBSCAN) for desired sheet**)
data = pd.read_excel(dbscan_file, sheet_name= 'B1_Ib_DBSCAN', usecols= [1, 2, 5, 37])

#Remove lateral localization accuracy points below 50nm (0.05 microns)
data_loc_acc = data.loc[data["Lateral Localization Accuracy"] <= 0.05]

#Sort data by Cluster ID
data_sorted = data_loc_acc.sort_values(by = ['Clusters - ID'], ascending = True )
data_sorted

,X Position [µm],Y Position [µm],Lateral Localization Accuracy,Clusters - ID
123,18.412582,11.790802,0.046826,2
162,18.440820,11.762816,0.029610,2
26,18.460686,11.766873,0.041973,2
28,18.410106,11.803893,0.022242,2
278,18.443359,11.799879,0.027378,2
...,...,...,...,...
661,17.547169,11.901045,0.040318,284
292,17.607776,11.957288,0.024288,284
673,17.641285,11.922378,0.038294,284
282,17.564518,11.935361,0.033214,284


In [4]:
# Load max projection image from raw nd2 video wih Fiji/ImageJ to maintain alignment
max_proj_file = 'MAX_20220526_CacHalo_JF646_L1_A2R67_000.nd2 - C=0.png'

Im = plt.imread(max_proj_file)

In [5]:
# Plot max projection image and all non-noise poits on top

# pop out to allow zoom, etc
%matplotlib qt  

pts_all = um_to_px( data[data['Clusters - ID']>=0].iloc[:,0:3].to_numpy() ) # pixels
pts_all_ids = data[data['Clusters - ID']>=0].iloc[:,3].to_numpy()

plt.figure(figsize=(12,12))
plt.imshow(Im.mean(axis=2), cmap = "gray")  # Plot greyscale
plt.scatter(pts_all[:,0],pts_all[:,1],5,pts_all_ids)
plt.axis('image')
plt.show()

In [6]:
# restore inline plotting
%matplotlib inline  

In [7]:
cluster_ids = data_sorted.iloc[:,3].unique()  # Check ID column
cluster_ids = cluster_ids[cluster_ids>=0]  # Reject noise cluster of -1 Column ID

#alpha_param = 500
alpha_param = 0.0124


cluster_stats = np.full( (len(cluster_ids),5), np.nan )
cluster_shapes = []
for kk, cid in enumerate(cluster_ids):
    print(f'Working on cluster {cid}')
    if cid>=0:
        coords = data[data['Clusters - ID']==cid].iloc[:,0:2].to_numpy()
        num_coords = len(coords)
        cluster_stats[kk,0] = cid  # Identifier
        cluster_stats[kk,1] = ash.get_alpha_area(coords, alpha_param)  # Area, um^2
        cluster_stats[kk,2] = 2*np.sqrt(cluster_stats[kk,1]/np.pi)  # Equivalent diameter, um^2
        cluster_stats[kk,3] = num_coords / cluster_stats[kk,1]  # Density, num/um^2
        cluster_stats[kk,4] = num_coords   # number of locs in cluster
        #Average the lateral loc accuracy column for each cluster ID?
        cluster_shapes.append( ash.get_alpha_shape(coords, alpha_param) ) # Full shape object

Working on cluster 2
Working on cluster 55
Working on cluster 125
Working on cluster 146
Working on cluster 211
Working on cluster 279
Working on cluster 284


In [8]:
# Don't do this
for stupid in cluster_shapes[0].find_optimal_alpha(1):
    print( stupid )

0.0004953892771927717
0.00053655326334591
0.0008944679192811337
0.0010570511613214196
0.0015687292661269127
0.001605645141610123
0.0019115560789220512
0.012443642060180055


In [9]:
cluster_shapes[1].set_alpha(500)
print( cluster_shapes[1].get_alpha() )


500.0


In [10]:
%matplotlib qt  

In [11]:
# Put segmented alpha_shpaes on top of map
plt.figure()
plt.imshow(Im[:,:,0], cmap = "gray")

for cur_shape in cluster_shapes:
    cur_shape.set_alpha(alpha_param)
    cluster_color = np.random.rand(1,3)
    for cur_edge in cur_shape.alpha_shape_edges():
        cur_segment = cur_shape.segment(cur_edge)
        print( cur_segment, cur_shape.classify(cur_edge) )
        X_from = cur_segment.source().x()
        X_to = cur_segment.target().x()
        Y_from = cur_segment.source().y()
        Y_to = cur_segment.target().y()
        # TODO: Assemble into numpy array
        X_px = um_to_px( np.array([X_from,X_to]) )
        Y_px = um_to_px( np.array([Y_from,Y_to]) )
        plt.plot(X_px,Y_px,color=cluster_color)
        plt.scatter(pts_all[:,0],pts_all[:,1],5,color=cluster_color)
        # TODO: Get points from alpha_shape for consistency
#plt.axis('equal')
plt.axis('image')


18.4026 11.7754 18.4138 11.7358 2
18.4232 11.7291 18.4675 11.7169 2
18.4675 11.7169 18.4728 11.7655 2
18.4346 11.8232 18.4013 11.8154 2
18.4728 11.7655 18.4607 11.8192 2
18.4013 11.8154 18.4026 11.7754 2
18.4138 11.7358 18.4232 11.7291 2
18.4607 11.8192 18.4528 11.8325 2
18.4528 11.8325 18.4346 11.8232 2
17.5194 11.4245 17.496 11.4466 2
17.3662 11.3367 17.3369 11.3248 2
17.3491 11.2323 17.3839 11.2403 2
17.4816 11.4268 17.453 11.401 2
17.336 11.2658 17.344 11.2397 2
17.3233 11.3012 17.336 11.2658 2
17.4467 11.3024 17.4882 11.3365 2
17.4009 11.3605 17.3662 11.3367 2
17.4882 11.3365 17.5184 11.4078 2
17.453 11.401 17.4086 11.3675 2
17.3839 11.2403 17.4467 11.3024 2
17.344 11.2397 17.3491 11.2323 2
17.4086 11.3675 17.4009 11.3605 2
17.5184 11.4078 17.5194 11.4245 2
17.496 11.4466 17.4816 11.4268 2
17.3369 11.3248 17.3233 11.3012 2
18.2909 10.8823 18.3102 10.8898 2
18.3102 10.8898 18.3469 10.9072 2
18.3613 11.0152 18.3196 11.0233 2
18.3469 10.9072 18.3641 10.9721 2
18.2457 10.8771 18.2773 

(-0.5, 255.5, 255.5, -0.5)

In [12]:
# Back of the envelope check that area is reasonable, given image
cluster_stats[0,1], (11.82-11.72)*(18.48-18.40)

(0.006122390233731345, 0.008000000000000156)

In [13]:
# TODO: Plot alpha shapes on top of points
        #Can we add in average value from localization accuracy column for each cluster?

In [14]:
cluster_stats_pd = pd.DataFrame(cluster_stats, columns = ['Cluster_ID','ClusterArea ( $\mu m^2$ )','ClusterDiameter ( $\mu m$ )','Density','# of Locs'])
cluster_stats_pd

,Cluster_ID,ClusterArea ( $\mu m^2$ ),ClusterDiameter ( $\mu m$ ),Density,# of Locs
0,2.0,0.006122,0.088291,6370.061122,39.0
1,55.0,0.017219,0.148069,9466.095003,163.0
2,125.0,0.013686,0.132005,7233.817994,99.0
3,146.0,0.016048,0.142944,10842.388905,174.0
4,211.0,0.011338,0.120150,4233.528488,48.0
5,279.0,0.021658,0.166061,13066.586924,283.0
6,284.0,0.016384,0.144432,9399.547333,154.0
